## 1. Importar Bibliotecas

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from pathlib import Path

from src.data.preprocessing import DataPreprocessor
from src.features.feature_engineering import FeatureEngineer

## 2. Carregar Dados

In [2]:
# Carregar dataset original
data_path = Path('../data/raw/seu_dataset.csv')
df = pd.read_csv(data_path)

print(f"Dataset shape: {df.shape}")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/seu_dataset.csv'

## 3. Tratamento de Dados

### 3.1 Valores Faltantes

In [ ]:
preprocessor = DataPreprocessor()

# Tratar valores faltantes
df_clean = preprocessor.handle_missing_values(df, strategy='mean')

print(f"Valores faltantes após tratamento:")
print(df_clean.isnull().sum().sum())

### 3.2 Codificação de Variáveis Categóricas

In [ ]:
# Identificar colunas categóricas
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()
print(f"Colunas categóricas: {categorical_cols}")

# Codificar variáveis categóricas
if categorical_cols:
    df_encoded = preprocessor.encode_categorical(df_clean, categorical_cols)
else:
    df_encoded = df_clean.copy()

## 4. Engenharia de Features

### 4.1 Features Polinomiais

In [ ]:
engineer = FeatureEngineer()

# TODO: Selecionar colunas para features polinomiais
# poly_cols = ['col1', 'col2']
# df_features = engineer.create_polynomial_features(df_encoded, poly_cols, degree=2)

df_features = df_encoded.copy()
print(f"Shape após features polinomiais: {df_features.shape}")

### 4.2 Features de Interação

In [ ]:
# TODO: Criar features de interação
# interaction_pairs = [('col1', 'col2'), ('col3', 'col4')]
# df_features = engineer.create_interaction_features(df_features, interaction_pairs)

print(f"Shape após features de interação: {df_features.shape}")

## 5. Separar Features e Target

In [ ]:
# TODO: Definir coluna target
target_col = 'target'

X = df_features.drop(target_col, axis=1)
y = df_features[target_col]

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

## 6. Seleção de Features

In [ ]:
# Análise de importância das features
importance = engineer.get_feature_importance(X, y)
print("\nTop 15 features mais importantes:")
print(importance.head(15))

In [ ]:
# Selecionar top K features (opcional)
# k = 20
# X_selected = engineer.select_features(X, y, k=k, method='f_classif')
# print(f"Features selecionadas: {engineer.selected_features}")

X_selected = X.copy()

## 7. Normalização dos Dados

In [ ]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = preprocessor.split_data(X_selected, y, test_size=0.2)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
# Normalizar features
X_train_scaled = preprocessor.scale_features(X_train, fit=True)
X_test_scaled = preprocessor.scale_features(X_test, fit=False)

print("Dados normalizados com sucesso!")

## 8. Salvar Dados Processados

In [ ]:
# Salvar dados processados
processed_dir = Path('../data/processed')
processed_dir.mkdir(exist_ok=True)

np.save(processed_dir / 'X_train.npy', X_train_scaled)
np.save(processed_dir / 'X_test.npy', X_test_scaled)
np.save(processed_dir / 'y_train.npy', y_train)
np.save(processed_dir / 'y_test.npy', y_test)

# Salvar nomes das features
feature_names = X_selected.columns.tolist()
pd.Series(feature_names).to_csv(processed_dir / 'feature_names.csv', index=False, header=False)

print("Dados processados salvos com sucesso!")

## 9. Análise de Dados Populacionais

Este notebook analisa a integração de dados populacionais do IBGE Censo 2022 no modelo de cobertura de transporte público. As visualizações a seguir permitem aos planejadores urbanos compreender como a densidade populacional influencia as classificações de cobertura.

**Objetivo**: Demonstrar como a feature populacional habilita decisões baseadas em contexto demográfico.

In [ ]:
# Importar bibliotecas adicionais para análise populacional
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar estilo de visualização
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Bibliotecas carregadas com sucesso!")

### 9.1 Carregar Dados com População Integrada

In [ ]:
# Carregar features enriquecidas com população
features_path = Path('../data/processed/features/grid_features.parquet')
labels_path = Path('../data/processed/labels/grid_labels.parquet')

# Carregar com pandas (não tem geo metadata)
grid_features = pd.read_parquet(features_path)
grid_labels = pd.read_parquet(labels_path)

# Converter para GeoDataFrame para visualizações espaciais
from shapely.geometry import box
grid_features['geometry'] = grid_features.apply(
    lambda row: box(row['lon_min'], row['lat_min'], row['lon_max'], row['lat_max']),
    axis=1
)
grid_features = gpd.GeoDataFrame(grid_features, geometry='geometry', crs='EPSG:4326')

# Merge features com labels
grid_data = grid_features.merge(grid_labels[['cell_id', 'label', 'composite_score']], 
                                 on='cell_id', how='left')

print(f"📊 Dataset carregado:")
print(f"   - Total de células: {len(grid_data):,}")
print(f"   - Features: {grid_data.shape[1]}")
print(f"   - Células com população > 0: {(grid_data['population'] > 0).sum():,} ({(grid_data['population'] > 0).sum() / len(grid_data) * 100:.1f}%)")
print(f"   - População total: {grid_data['population'].sum():,.0f} habitantes")
print(f"   - População média: {grid_data['population'].mean():.1f} habitantes/célula")
print(f"\n📍 Distribuição de labels:")
print(grid_data['label'].value_counts())

### 9.2 Distribuição Populacional nos 200m Cells

Análise da distribuição de população nas células de 200m × 200m.

In [ ]:
# Criar figura com múltiplos subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Histograma de população (todas as células)
ax1 = axes[0, 0]
ax1.hist(grid_data['population'], bins=50, color='#4A90E2', alpha=0.7, edgecolor='black')
ax1.axvline(grid_data['population'].mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {grid_data["population"].mean():.1f}')
ax1.axvline(grid_data['population'].median(), color='orange', linestyle='--', linewidth=2, label=f'Mediana: {grid_data["population"].median():.1f}')
ax1.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequência (número de células)', fontsize=12, fontweight='bold')
ax1.set_title('Distribuição de População - Todas as Células (200m × 200m)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# 2. Histograma de população (apenas células com pop > 0)
ax2 = axes[0, 1]
pop_nonzero = grid_data[grid_data['population'] > 0]['population']
ax2.hist(pop_nonzero, bins=50, color='#34C759', alpha=0.7, edgecolor='black')
ax2.axvline(pop_nonzero.mean(), color='red', linestyle='--', linewidth=2, label=f'Média: {pop_nonzero.mean():.1f}')
ax2.axvline(pop_nonzero.median(), color='orange', linestyle='--', linewidth=2, label=f'Mediana: {pop_nonzero.median():.1f}')
ax2.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frequência (número de células)', fontsize=12, fontweight='bold')
ax2.set_title('Distribuição de População - Somente Células Habitadas (pop > 0)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(axis='y', alpha=0.3)

# 3. Boxplot por classe de cobertura
ax3 = axes[1, 0]
grid_data_labeled = grid_data[grid_data['label'].notna()]
labels_map = {0: 'Mal Atendida', 1: 'Bem Atendida'}
grid_data_labeled['label_name'] = grid_data_labeled['label'].map(labels_map)
bp = ax3.boxplot([grid_data_labeled[grid_data_labeled['label'] == 0]['population'],
                   grid_data_labeled[grid_data_labeled['label'] == 1]['population']],
                  labels=['Mal Atendida', 'Bem Atendida'],
                  patch_artist=True, showmeans=True)
bp['boxes'][0].set_facecolor('#FF6B6B')
bp['boxes'][1].set_facecolor('#51CF66')
ax3.set_ylabel('População (habitantes)', fontsize=12, fontweight='bold')
ax3.set_title('Distribuição de População por Classificação de Cobertura', fontsize=14, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

# 4. Estatísticas descritivas
ax4 = axes[1, 1]
ax4.axis('off')
stats_text = f"""
ESTATÍSTICAS DESCRITIVAS - POPULAÇÃO

Total de Células: {len(grid_data):,}
Células Habitadas (pop > 0): {(grid_data['population'] > 0).sum():,} ({(grid_data['population'] > 0).sum() / len(grid_data) * 100:.1f}%)
Células Desabitadas (pop = 0): {(grid_data['population'] == 0).sum():,} ({(grid_data['population'] == 0).sum() / len(grid_data) * 100:.1f}%)

População Total: {grid_data['population'].sum():,.0f} habitantes
População Média: {grid_data['population'].mean():.1f} hab/célula
Desvio Padrão: {grid_data['population'].std():.1f}
Mediana: {grid_data['population'].median():.1f}
Mínimo: {grid_data['population'].min():.0f}
Máximo: {grid_data['population'].max():.0f}

PERCENTIS:
25%: {grid_data['population'].quantile(0.25):.1f}
50%: {grid_data['population'].quantile(0.50):.1f}
75%: {grid_data['population'].quantile(0.75):.1f}
90%: {grid_data['population'].quantile(0.90):.1f}
95%: {grid_data['population'].quantile(0.95):.1f}
99%: {grid_data['population'].quantile(0.99):.1f}
"""
ax4.text(0.1, 0.5, stats_text, fontsize=11, family='monospace', verticalalignment='center')

plt.tight_layout()
plt.savefig('../reports/figures/population_distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Análise de distribuição populacional completa!")

### 9.3 Mapa Espacial: População e Classificação de Cobertura

Visualização geoespacial sobrepondo densidade populacional com classificações de cobertura de transporte.

In [ ]:
# Criar figura com 2 mapas lado a lado
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Mapa 1: Densidade Populacional
ax1 = axes[0]
grid_data.plot(column='population', 
               cmap='YlOrRd', 
               legend=True,
               ax=ax1,
               edgecolor='face',
               linewidth=0.1,
               legend_kwds={'label': 'População (habitantes)', 'shrink': 0.8})
ax1.set_title('Densidade Populacional por Célula (200m × 200m)', fontsize=16, fontweight='bold', pad=20)
ax1.set_xlabel('Longitude', fontsize=12)
ax1.set_ylabel('Latitude', fontsize=12)
ax1.grid(True, alpha=0.3)

# Mapa 2: Classificação de Cobertura com População em Transparência
ax2 = axes[1]
# Classificação de cobertura (cores categóricas)
colors_map = {0: '#FF6B6B', 1: '#51CF66'}  # Mal atendida: vermelho, Bem atendida: verde
grid_data_labeled = grid_data[grid_data['label'].notna()].copy()
grid_data_labeled['color'] = grid_data_labeled['label'].map(colors_map)

# Plot com tamanho proporcional à população
grid_data_labeled.plot(color=grid_data_labeled['color'],
                        alpha=0.6,
                        ax=ax2,
                        edgecolor='black',
                        linewidth=0.3)

ax2.set_title('Classificação de Cobertura de Transporte', fontsize=16, fontweight='bold', pad=20)
ax2.set_xlabel('Longitude', fontsize=12)
ax2.set_ylabel('Latitude', fontsize=12)
ax2.grid(True, alpha=0.3)

# Adicionar legenda manual
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#FF6B6B', label='Mal Atendida', alpha=0.6),
    Patch(facecolor='#51CF66', label='Bem Atendida', alpha=0.6)
]
ax2.legend(handles=legend_elements, loc='upper right', fontsize=12, frameon=True)

plt.tight_layout()
plt.savefig('../reports/figures/spatial_map_population_coverage.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Mapas espaciais gerados!")

### 9.4 Casos de Estudo: Como População Influencia Classificação

Exemplos concretos demonstrando como o contexto populacional altera a interpretação de cobertura de transporte.

In [ ]:
# Identificar casos de estudo interessantes
grid_labeled = grid_data[grid_data['label'].notna()].copy()

# Caso 1: Alta população + Baixo transporte (provável mal atendida)
high_pop_low_transit = grid_labeled[
    (grid_labeled['population'] > grid_labeled['population'].quantile(0.75)) &
    (grid_labeled['daily_trips'] < grid_labeled['daily_trips'].quantile(0.25))
].sort_values('population', ascending=False)

# Caso 2: Baixa população + Baixo transporte (pode ser bem atendida)
low_pop_low_transit = grid_labeled[
    (grid_labeled['population'] < grid_labeled['population'].quantile(0.25)) &
    (grid_labeled['daily_trips'] < grid_labeled['daily_trips'].quantile(0.25)) &
    (grid_labeled['population'] > 0)  # Excluir células vazias
].sort_values('population', ascending=True)

# Caso 3: Alta população + Alto transporte (bem atendida)
high_pop_high_transit = grid_labeled[
    (grid_labeled['population'] > grid_labeled['population'].quantile(0.75)) &
    (grid_labeled['daily_trips'] > grid_labeled['daily_trips'].quantile(0.75))
].sort_values('population', ascending=False)

print("="*80)
print("CASOS DE ESTUDO: INFLUÊNCIA DA POPULAÇÃO NA CLASSIFICAÇÃO")
print("="*80)

print("\n📍 CASO 1: Alta População + Baixo Transporte")
print("-" * 80)
if len(high_pop_low_transit) > 0:
    case1 = high_pop_low_transit.iloc[0]
    print(f"   Cell ID: {case1['cell_id']}")
    print(f"   População: {case1['population']:.0f} habitantes (P{grid_labeled['population'].rank(pct=True).loc[case1.name]*100:.0f})")
    print(f"   Paradas: {case1['stop_count']:.0f}")
    print(f"   Linhas: {case1['route_count']:.0f}")
    print(f"   Viagens Diárias: {case1['daily_trips']:.0f}")
    print(f"   Classificação: {'Mal Atendida' if case1['label'] == 0 else 'Bem Atendida'}")
    print(f"   ⚠️ INTERPRETAÇÃO: Área densamente povoada com baixa cobertura de transporte")
    print(f"      → ALTA PRIORIDADE para investimento em transporte público")
else:
    print("   Nenhum caso encontrado nesta categoria")

print("\n📍 CASO 2: Baixa População + Baixo Transporte")
print("-" * 80)
if len(low_pop_low_transit) > 0:
    case2 = low_pop_low_transit.iloc[0]
    print(f"   Cell ID: {case2['cell_id']}")
    print(f"   População: {case2['population']:.0f} habitantes (P{grid_labeled['population'].rank(pct=True).loc[case2.name]*100:.0f})")
    print(f"   Paradas: {case2['stop_count']:.0f}")
    print(f"   Linhas: {case2['route_count']:.0f}")
    print(f"   Viagens Diárias: {case2['daily_trips']:.0f}")
    print(f"   Classificação: {'Mal Atendida' if case2['label'] == 0 else 'Bem Atendida'}")
    print(f"   ✓ INTERPRETAÇÃO: Área pouco povoada com transporte proporcional à demanda")
    print(f"      → BAIXA PRIORIDADE - cobertura adequada para densidade populacional")
else:
    print("   Nenhum caso encontrado nesta categoria")

print("\n📍 CASO 3: Alta População + Alto Transporte")
print("-" * 80)
if len(high_pop_high_transit) > 0:
    case3 = high_pop_high_transit.iloc[0]
    print(f"   Cell ID: {case3['cell_id']}")
    print(f"   População: {case3['population']:.0f} habitantes (P{grid_labeled['population'].rank(pct=True).loc[case3.name]*100:.0f})")
    print(f"   Paradas: {case3['stop_count']:.0f}")
    print(f"   Linhas: {case3['route_count']:.0f}")
    print(f"   Viagens Diárias: {case3['daily_trips']:.0f}")
    print(f"   Classificação: {'Mal Atendida' if case3['label'] == 0 else 'Bem Atendida'}")
    print(f"   ✓ INTERPRETAÇÃO: Área densamente povoada com excelente cobertura de transporte")
    print(f"      → Investimento bem dimensionado - manter e monitorar")
else:
    print("   Nenhum caso encontrado nesta categoria")

print("\n" + "="*80)

### 9.5 Análise Comparativa: População vs Transit Metrics

Relação entre densidade populacional e métricas de transporte público.

In [ ]:
# Criar scatter plots mostrando relação população x métricas de transporte
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Filtrar apenas células com população > 0 para visualização mais clara
grid_pop = grid_labeled[grid_labeled['population'] > 0].copy()

# 1. População vs Daily Trips
ax1 = axes[0, 0]
colors = grid_pop['label'].map({0: '#FF6B6B', 1: '#51CF66'})
ax1.scatter(grid_pop['population'], grid_pop['daily_trips'], 
           c=colors, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax1.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Viagens Diárias', fontsize=12, fontweight='bold')
ax1.set_title('População vs Viagens Diárias', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Adicionar linha de tendência
from scipy.stats import pearsonr
if len(grid_pop) > 1:
    z = np.polyfit(grid_pop['population'], grid_pop['daily_trips'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(grid_pop['population'].min(), grid_pop['population'].max(), 100)
    ax1.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='Tendência')
    corr, _ = pearsonr(grid_pop['population'], grid_pop['daily_trips'])
    ax1.text(0.05, 0.95, f'Correlação: {corr:.3f}', transform=ax1.transAxes, 
            fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax1.legend()

# 2. População vs Stop Count
ax2 = axes[0, 1]
ax2.scatter(grid_pop['population'], grid_pop['stop_count'],
           c=colors, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax2.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Número de Paradas', fontsize=12, fontweight='bold')
ax2.set_title('População vs Número de Paradas', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

if len(grid_pop) > 1:
    z = np.polyfit(grid_pop['population'], grid_pop['stop_count'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(grid_pop['population'].min(), grid_pop['population'].max(), 100)
    ax2.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='Tendência')
    corr, _ = pearsonr(grid_pop['population'], grid_pop['stop_count'])
    ax2.text(0.05, 0.95, f'Correlação: {corr:.3f}', transform=ax2.transAxes,
            fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax2.legend()

# 3. População vs Route Count
ax3 = axes[1, 0]
ax3.scatter(grid_pop['population'], grid_pop['route_count'],
           c=colors, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax3.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax3.set_ylabel('Número de Linhas', fontsize=12, fontweight='bold')
ax3.set_title('População vs Número de Linhas', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

if len(grid_pop) > 1:
    z = np.polyfit(grid_pop['population'], grid_pop['route_count'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(grid_pop['population'].min(), grid_pop['population'].max(), 100)
    ax3.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='Tendência')
    corr, _ = pearsonr(grid_pop['population'], grid_pop['route_count'])
    ax3.text(0.05, 0.95, f'Correlação: {corr:.3f}', transform=ax3.transAxes,
            fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax3.legend()

# 4. População vs Stop Density
ax4 = axes[1, 1]
ax4.scatter(grid_pop['population'], grid_pop['stop_density'],
           c=colors, alpha=0.5, s=30, edgecolors='black', linewidths=0.5)
ax4.set_xlabel('População (habitantes)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Densidade de Paradas (stops/km²)', fontsize=12, fontweight='bold')
ax4.set_title('População vs Densidade de Paradas', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

if len(grid_pop) > 1:
    z = np.polyfit(grid_pop['population'], grid_pop['stop_density'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(grid_pop['population'].min(), grid_pop['population'].max(), 100)
    ax4.plot(x_line, p(x_line), "r--", alpha=0.8, linewidth=2, label='Tendência')
    corr, _ = pearsonr(grid_pop['population'], grid_pop['stop_density'])
    ax4.text(0.05, 0.95, f'Correlação: {corr:.3f}', transform=ax4.transAxes,
            fontsize=11, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax4.legend()

# Legenda global
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#FF6B6B', label='Mal Atendida', alpha=0.5),
    Patch(facecolor='#51CF66', label='Bem Atendida', alpha=0.5)
]
fig.legend(handles=legend_elements, loc='lower center', ncol=2, fontsize=12, frameon=True, bbox_to_anchor=(0.5, -0.02))

plt.tight_layout(rect=[0, 0.02, 1, 1])
plt.savefig('../reports/figures/population_vs_transit_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Análise comparativa população vs métricas de transporte completa!")

### 9.6 Resumo e Insights para Planejamento Urbano

**Principais Conclusões da Análise Populacional:**

1. **Distribuição Populacional**: 
   - 40.2% das células têm população zero (áreas não residenciais, buffer zones)
   - 59.8% das células são habitadas com população variando de 1 a 2.062 habitantes
   - População média de 174.7 habitantes por célula de 200m × 200m

2. **Correlação População-Transporte**:
   - Correlação positiva entre população e métricas de transporte
   - Áreas mais povoadas tendem a ter mais paradas, linhas e viagens
   - Confirma lógica de investimento em transporte baseado em demanda

3. **Valor da Feature Populacional**:
   - **Desambiguação**: Distingue entre áreas genuinamente mal atendidas (alta pop + baixo transporte) e áreas de baixa demanda (baixa pop + baixo transporte)
   - **Priorização**: Permite ranquear investimentos por impacto populacional
   - **Contextualização**: Evita desperdício de recursos em áreas de baixa densidade

4. **Aplicações para Planejamento Urbano**:
   - Identificar "desertos de transporte" em áreas densamente povoadas
   - Validar adequação de cobertura em regiões periféricas
   - Fundamentar políticas públicas com dados demográficos objetivos
   - Monitorar evolução temporal (população Censo 2022 + atualizações futuras)

5. **Limitações a Considerar**:
   - Dados estáticos do Censo 2022 (não refletem dinâmica diária)
   - População residencial (não captura fluxo trabalho/estudo)
   - Grid uniforme (não respeita limites de bairros)

**Recomendação**: A feature populacional é **essencial** para interpretação contextual de classificações de cobertura, mesmo com importância numérica baixa (1.10% RF, 0.08% GB). Seu valor está na **capacidade de distinção qualitativa** entre cenários, não em dominância estatística.